# FEATURE EXTRACTION TO CSV FILE

In [1]:
import re, os, sys
from bs4 import BeautifulSoup
import pprint
from urllib.parse import urlparse
import email
from IPy import IP
import email.header
from email.parser import BytesParser, Parser
from email.policy import default
import csv
from collections import Counter
from html.parser import HTMLParser
from spamassassin_client import SpamAssassin
import whois
import validators

In [2]:
'''to remove warnings'''
import warnings
warnings.filterwarnings("ignore")

CHANGE THE DIRECTORY TO CHANGE TARGET DIRECTORY

Locations:
- /Datasets/non_phishing
- /Datasets/phishing
- /Datasets/phishing_bad
- /Datasets/non_phishing_bad
- /IWSPA-PA Dataset/legit
- /IWSPA-PA Dataset/phish
- /Test Datasets/PhishingNew
- /Test Datasets/Legitimate

In [3]:
'''CHANGE THIS'''
os.chdir('Datasets/xd')
phishing_2016 = os.listdir()

In [4]:
for x in phishing_2016:
    print("{}".format(x))

'Any thoughts on this Humble Bundle of Python...' - Reddit (noreply@redditmail.com) - 2020-07-02 1043.eml
'Help  partial string filtering on pandas dat...' - Reddit (noreply@redditmail.com) - 2020-06-21 1315.eml
'I get 2 weeks free AND a magazine, too ” You do. - 'America's Test Kitchen' (News@email.americastestkitchen.com) - 2020-06-23 1915.eml
7-Day Flash Sale- SGD45! - 'the ENTERTAINER' (info@theentertainerme.com) - 2020-07-02 1859.eml
Arboros Dragon, Sephirot on Cardfight!! Vanguard Wiki has been edited by HellfireIfrit - FANDOM (community@fandom.com) - 2020-06-23 2103.eml
As a beginner in AI, what tool should I focus to learn  Keras, TensorFlow or PyTorch...  - Quora Digest (digest-noreply@quora.com) - 2020-06-20 1446.eml
Britain's Future Commando Force - 'Telegraph Editor' (telegrapheditor@email3.telegraph.co.uk) - 2020-06-30 0009.eml
Check out this week’s most read articles - 'The Telegraph' (thetelegraph@email3.telegraph.co.uk) - 2020-06-20 2128.eml
Congratulate Yeow Kee Tan fo

# Feature Extraction (Body, Email)

In [5]:
# FUNCTIONS
def ExtractEmail(filename):
    fp = open(filename, 'rb')
    mail = fp.read()
    msg = email.message_from_bytes(mail)
    fp.close()
    return msg
    
def EmailParser(filename):
    fp = open(filename, 'rb')
    headers = BytesParser(policy=default).parse(fp)
    fp.close()
    return headers

def ExtractBody(msg):
    content = ""
    if msg.is_multipart():
        for payload in msg.get_payload():
            content += str(payload.get_payload())
    else:
        content += str(msg.get_payload())
    return content

def CheckAttachment(msg):
    attachment = 0
    for part in msg.walk():
        if part.get_content_maintype() == 'multipart':
            continue
        if part.get('Content-Disposition') is None:
            continue
    fileName = part.get_filename()
    if bool(fileName):
        attachment = 1
    return attachment

In [6]:
def SpamScore(filename):
    #print("{}".format(filename))
    spamScore = 0
    try:
        with open(filename,"rb") as f:      
            assassin = SpamAssassin(f.read())
            spamScore = assassin.get_score()
        print("The spam score is {}".format(spamScore))
    except:
        spamScore = 0
        print(filename)
    return spamScore

def checkSpam(filename):
    spamCheck = 0
    with open(filename,"rb") as f:            
        assassin = SpamAssassin(f.read())
        spamCheck = assassin.is_spam()
    print("it is a {}".format(spamCheck))
    return spamCheck

In [7]:
def checkSenderReplyDiff(header):
    senderReplyDiff = 1
    try:
        if header['from'] == header['to']:
            senderReplyDiff = 0
    except:
        senderReplyDiff = 1
    return senderReplyDiff

def subj_reply(header):
    try:
        subj_reply = header['subject'].lower().startswith("re:")
    except:
        subj_reply = ""
    return subj_reply

def subj_forward(header):
    try:
        subj_forward = header['subject'].lower().startswith("fwd:")
    except:
        subj_forward = ""
    return subj_forward

def subj_noOfWords(header):
    try:
        subj_noOfWords = len(header['subject'].split())
    except:
        subj_noOfWords = 0
    return subj_noOfWords

def subj_noOfChar(header):
    try:
        subj_noOfChar = len(header['subject'])
    except:
        subj_noOfChar = 0
    return subj_noOfChar

def subj_verify(header):
    try:
        subj_verify = "verify" in header['subject'].lower()
    except:
        subj_verify = ""
    return subj_verify

def subj_debit(header):
    try:
        subj_debit = "debit" in header['subject'].lower()
    except:
        subj_debit = ""
    return subj_debit

def subj_bank(header):
    try:
        subj_bank = "bank" in header['subject'].lower()
    except:
        subj_bank = ""
    return subj_bank

In [8]:
emails = []
header = []
spamScore = []
#spamCheck = []
#sender_emails = []
#receiver_emails = []
subject = []
SenderReplyDiff = []
subjReply = []
subjForward = []
subjNoOfWords = []
subjNoOfChar = []
subjVerify = []
subjDebit = []
subjBank = []

for filename in phishing_2016:
    emails.append(ExtractEmail(filename))
    header = EmailParser(filename)
    spamScore.append(SpamScore(filename))
    #spamCheck.append(checkSpam(filename))
    #sender_emails.append(header['from'])
    #receiver_emails.append(header['to'])
    subject.append(header['subject'])
    SenderReplyDiff.append(checkSenderReplyDiff(header))
    subjReply.append(subj_reply(header))
    subjForward.append(subj_forward(header))
    subjNoOfWords.append(subj_noOfWords(header))
    subjNoOfChar.append(subj_noOfChar(header))
    subjVerify.append(subj_verify(header))
    subjDebit.append(subj_debit(header))
    subjBank.append(subj_bank(header))

The spam score is -1.0
The spam score is -1.0
The spam score is 1.3
The spam score is -0.6
The spam score is -1.7
The spam score is -1.7
The spam score is -1.7
The spam score is -1.7
The spam score is -0.9
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -0.4
The spam score is -0.4
The spam score is -1.7
The spam score is -1.7
The spam score is -0.9
The spam score is -5.9
The spam score is 0.1
The spam score is -1.7
The spam score is -0.9
The spam score is -0.9
The spam score is -1.1
The spam score is -1.7
The spam score is -1.4
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam score is -1.0
The spam scor

In [9]:
body_contents = []
checkAttachments = []

for email in emails:
    body_contents.append(ExtractBody(email))
    checkAttachments.append(CheckAttachment(email))
    #out.println("{}".format(email))

# Check for Scripts

In [10]:
def ScriptCheck(body_content):
    script_scripts = False
    try:
        yikes = 0
        soup = BeautifulSoup(body_content,'lxml')
        script  = soup.find_all('script')
        if len(script) > 0:
            script_scripts = True
    except:
        script_scripts = False
    return script_scripts

In [11]:
def check_popups(body_content):
    script_popups = 0
    if ScriptCheck(body_content):
        soup = BeautifulSoup(body_content)
        for script in soup.find_all('script'):
            if "window.open" in str(script.contents):
                script_popups = 1
    return script_popups

def check_javascript(body_content):
    script_javascript = 0
    if ScriptCheck(body_content):
        soup = BeautifulSoup(body_content)
        for script in soup.find_all('script'):
            if script.get('type') is not None:
                if "text/javascript" in script.get('type'):
                    script_javascript = 1
    return script_javascript

def noOfOnClickEvents(body_content):
    noOnClick = 0
    if ScriptCheck(body_content):
        soup = BeautifulSoup(body_content)
        codes = soup.find_all('button',{"onclick":True})
        noOnClick = len(codes)
    return noOnClick

In [12]:
script_exist = []
checkPopup = []
checkJS = []
NoOnClick = []

for body in body_contents:
    script_exist.append(ScriptCheck(body))
    checkPopup.append(check_popups(body))
    checkJS.append(check_javascript(body))
    NoOnClick.append(noOfOnClickEvents(body))

# Check for HTML, Forms, Phishing Terms

In [13]:
def HTMLCheck(body_content):
    body_html = bool(BeautifulSoup(body_content, "lxml").find())
    return body_html

def FormCheck(body_content):
    body_forms = bool(BeautifulSoup(body_content, "lxml").find("form"))
    return body_forms

def checkFunctionWords(body_content):
    body_noFunctionWords = 0
    wordlist = re.sub("[^A-Za-z]", " ", body_content.strip()).lower().split()
    function_words = ["account", "access", "bank", "credit", "click", "identity", "inconvenience", "information", "limited", 
                      "log", "minutes", "password", "recently", "risk", "social", "security", "service", "suspended", "deactivate"]
    for word in function_words:
        body_noFunctionWords += wordlist.count(word)
    return body_noFunctionWords

def checkNoWords(body_content):
    body_NoWords = len(body_content.split())
    return body_NoWords

def checkNoChars(body_content):
    body_NoChars = len(body_content) - body_content.count(' ') - body_content.count('\n')
    return body_NoChars

def checkRichness(noWords, noChars):
    body_richness = 0
    try:
        body_richness = noWords/noChars
    except:
        body_richness = 0
    return body_richness

def checkSuspension(body_content):
    body_suspension = "suspension" in body_content.lower()
    return body_suspension

def checkVerifyAcc(body_content):
    phrase = "verifyyouraccount"
    content = re.sub(r"[^A-Za-z]", "", body_content.strip()).lower()
    body_verifyYourAccount = phrase in content
    return body_verifyYourAccount

def checkDear(body_content):
    body_dear = "dear" in body_content.lower()
    return body_dear

def uniqueWords(body_content):
    words = set(body_content)
    count = len(words)
    return count

In [14]:
html_exist = []
form_exist = []
noFunctionWords = []
noWords = []
noChars = []
check_suspend = []
check_verify = []
check_dear = []
unique_Words = []

for body in body_contents:
    html_exist.append(HTMLCheck(str(body)))
    form_exist.append(FormCheck(str(body)))
    noFunctionWords.append(checkFunctionWords(str(body)))
    noWords.append(checkNoWords(str(body)))
    noChars.append(checkNoChars(str(body)))
    check_suspend.append(checkSuspension(str(body)))
    check_verify.append(checkVerifyAcc(str(body)))
    check_dear.append(checkDear(str(body)))
    unique_Words.append(uniqueWords(str(body)))

In [15]:
Richness = []

for x in range(0,len(noWords)):
    Richness.append(checkRichness(noWords[x],noChars[x]))

# Check for Links

In [16]:
def getLinks(body_content):
    links = []
    content = []
    soup = BeautifulSoup(body_content, "lxml")
    for link in soup.findAll('a'):
        links.append(link.get("href"))
        content.append(link.text)
    return links, content
    
def getURLs(body_content):
    urls = re.findall(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", body_content)
    return urls

def get_domain(url):
    domain = None
    if url:
        if u'@' not in str(url):
            parsed_uri = urlparse(url)
            domain = '{uri.netloc}'.format(uri=parsed_uri)
            if domain.startswith("www."):
                return domain[4:]
    return domain

In [17]:
def check_ipaddr(links):
    no_ipaddr = 1
    ip_addr = []
    for link in links:
        link_address = get_domain(link)
        if ":" in str(link_address):
            link_address = link_address[:link_address.index(":")]
        try:
            IP(link_address)
            no_ipaddr = 0
            break
        except:
            continue
    return no_ipaddr

def check_atsym(links):
    no_atsym = 1
    ip_addr = []
    for link in links:
        if u'@' in str(link):
            no_atsym = 0
            break
    return no_atsym

def check_doubleslash(links):
    no_doubleslash = 1
    for link in links:
        xd = str(link)[10:]
        if u'//' in xd:
            no_doubleslash = 0
    return no_doubleslash

def check_urlshorten(links):
    no_urlshorten = 1
    for link in links:
        if u'bit.ly' in str(link):
            no_urlshorten = 0
        elif u'goo.gl' in str(link):
            no_urlshorten = 0
        elif u'tinyurl.com' in str(link):
            no_urlshorten = 0
        elif u'adf.ly' in str(link):
            no_urlshorten = 0
    return no_urlshorten

def check_img(body_content):
    soup = BeautifulSoup(body_content)
    ImgLinks = soup.findAll('img')
    noOfImgLinks = len(ImgLinks)
    return noOfImgLinks

def check_port(links):
    no_diffport = 1
    for link in links:
        link_address = get_domain(link)
        if ":" in str(link_address):
            port = link_address[link_address.index(":"):][1:]
            if str(port) != str(80):
                no_diffport = 0
                break
    return no_diffport

def modalDomain(urls):
    if urls:
        modal_url = max(set(urls), key = urls.count)
        return modal_url
    else:
        return None
    
def check_NoOfDots(links):
    noOfDots = 0
    avg = 0
    for link in links:
        if link is not None:
            noOfDots += link.count('.')
    try:
        avg = noOfDots/len(links)
    except:
        avg = 0
    return avg
        
def avgDomainLength(links):
    leng = 0
    avgLen = 0
    for link in links:
        if link is not None:
            leng += len(link)
    try:
        avgLen = leng/len(links)
    except:
        avgLen = 0
    return avgLen

def check_ContentURL(links,contents):
    checkContentURL = 1
    for x in range(0,len(contents)):
        if validators.domain(contents[x]) == True:
            if links[x] != contents[x]:
                checkContentURL = 0
    return checkContentURL

def Whois (links):
    checkWhois = 0
    domain = ''
    for link in links:
        domain = get_domain(link)
        try:
            query = whois.query(domain)
            date = query.creation_date
            # CHECK IF DATE IF 60 DAYS OFEMAIL SENDING DATE AFTER CREATING WEBSITE
        except:
            continue
    return checkWhois

In [18]:
noOfURLs = []
noOfLinks = []

for body in body_contents:
    urls = getURLs(body)
    noOfURLs.append(len(urls))
    links, content = getLinks(body)
    noOfLinks.append(len(links))

In [19]:
checkIP = []
checkAtSym = []
checkdoubleslash = []
checkURLShorten = []
checkImg = []
checkPort = []
modalURL = []
checkNoDots = []
avg_DomainLength = []
checkContentURL = []

for body in body_contents:
    links, content = getLinks(body)
    urls = getURLs(body)
    checkIP.append(check_ipaddr(links))
    checkAtSym.append(check_atsym(links))
    checkdoubleslash.append(check_doubleslash(links))
    checkURLShorten.append(check_urlshorten(links))
    checkImg.append(check_img(body))
    checkPort.append(check_port(links))
    modalURL.append(modalDomain(urls))
    checkNoDots.append(check_NoOfDots(links))
    avg_DomainLength.append(avgDomainLength(links))
    checkContentURL.append(check_ContentURL(links, content))

# Modal Domains

In [20]:
def get_email_domain(string):
    domain = re.search("@[\w.]+", string)
    if domain is None:
        return None
    return str(domain.group())[1:]

In [21]:
def checkModalDomainHeader (email_addr, modalURL):
    send_domain = get_email_domain(email_addr)
    modal_domain = get_domain(modalURL)
    
    send_nonModalSenderDomain = 0
    if str(modalURL) != "None":
        if send_domain != modal_domain:
            send_nonModalSenderDomain = 1
    return send_nonModalSenderDomain

def checkHere(links, contents, modalURL):
    checkHere = 1
    for x in range(0,len(links)):
        if contents[x] is not None:
            if 'here' in contents[x].lower():
                domain = get_domain(links[x])
                if domain != 'here':
                    checkHere = 0
    return checkHere

In [22]:
checkModal = []
modalURLs = []

for x in range(0,len(phishing_2016)):
    modalURLs.append(modalURL[x])
    header = EmailParser(filename)
    checkModal.append(checkModalDomainHeader(header['from'],modalURL[x]))

In [23]:
check_Here = []

for x in range(0,len(body_contents)):
    links, contents = getLinks(body_contents[x])
    check_Here.append(checkHere(links, contents, modalURLs[x]))

# Write to CSV

REMEMBER TO CHANGE TARGET NAME

In [24]:
'''features_list = ['checkdoubleslash']

os.chdir('../../CSV')

with open('phishing_dataset_6.csv','w',newline='') as result_file:
    writer = csv.writer(result_file)
    writer.writerow(['checkdoubleslash'])
    result_file.flush()

    for x in range(0,len(emails)):
        print("{}".format(x))
        writer.writerow([checkdoubleslash[x]])
        result_file.flush()'''
        

'features_list = [\'checkdoubleslash\']\n\nos.chdir(\'../../CSV\')\n\nwith open(\'phishing_dataset_6.csv\',\'w\',newline=\'\') as result_file:\n    writer = csv.writer(result_file)\n    writer.writerow([\'checkdoubleslash\'])\n    result_file.flush()\n\n    for x in range(0,len(emails)):\n        print("{}".format(x))\n        writer.writerow([checkdoubleslash[x]])\n        result_file.flush()'

In [25]:
features_list = ['spamScore','SenderReplyDiff','checkAttachments','subjReply','subjForward','subjNoOfWords','subjNoOfChar','subjVerify','subjDebit',
                 'subjBank','script_exist','checkPopup','checkJS','NoOnClick','html_exist','form_exist','noFunctionWords',
                 'noWords','noChars','check_suspend','check_verify','check_dear','unique_Words','Richness','noOfURLs','noOfLinks','checkIP','checkAtSym','checkdoubleslash',
                 'checkURLShorten','checkImg','checkPort','checkNoDots','avg_DomainLength','checkContentURL','checkModal','check_Here']

os.chdir('../../CSV')

with open('non_phishing_testing_37.csv','w',newline='') as result_file:
    writer = csv.writer(result_file)
    writer.writerow(['spamScore','SenderReplyDiff','checkAttachments','subjReply','subjForward','subjNoOfWords','subjNoOfChar','subjVerify','subjDebit',
                 'subjBank','script_exist','checkPopup','checkJS','NoOnClick','html_exist','form_exist','noFunctionWords',
                 'noWords','noChars','check_suspend','check_verify','check_dear','unique_Words','Richness','noOfURLs','noOfLinks','checkIP','checkAtSym','checkdoubleslash',
                 'checkURLShorten','checkImg','checkPort','checkNoDots','avg_DomainLength','checkContentURL','checkModal','check_Here','Phishing',])
    result_file.flush()

    for x in range(0,len(emails)):
        print("{}".format(x))
        
        writer.writerow([spamScore[x],SenderReplyDiff[x],checkAttachments[x],subjReply[x],subjForward[x],subjNoOfWords[x],subjNoOfChar[x],subjVerify[x],subjDebit[x],
                     subjBank[x],script_exist[x],checkPopup[x],checkJS[x],NoOnClick[x],html_exist[x],form_exist[x],noFunctionWords[x],
                     noWords[x],noChars[x],check_suspend[x],check_verify[x],check_dear[x],unique_Words[x],Richness[x],noOfURLs[x],noOfLinks[x],checkIP[x],checkAtSym[x],checkdoubleslash[x],
                     checkURLShorten[x],checkImg[x],checkPort[x],checkNoDots[x],avg_DomainLength[x],checkContentURL[x],checkModal[x],check_Here[x],'0',])
        result_file.flush()
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
